# Apply e Tabelle pivot

In [12]:
import pandas as pd

## Apply

Il metodo `apply` permette di applicare una qualunque funzione a tutte le righe (o, più raramente, le colonne) di un  DataFrame o di una Serie. La funzione riceve direttamente una riga del DataFrame.

Una forma più sofisticata permette di applicare una funzione che riceve in input un intero DataFrame.

Come primo esempio, costruiamo una nuova variabile `pericolo` che corrisponde al numero di incidenti, più il numero di feriti moltiplicato per 100. Il primo passo è costruire la funzione che calcola tale indice.

In [43]:
def calcola_p(riga):
    return riga['Incidenti'] + riga['Feriti'] * 100

Poi si può usare la `apply`

In [44]:
incidenti['pericolo'] = incidenti.apply(calcola_p, axis = 1)
incidenti.head()

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo,pericolo
0,2001,69,50,1,1,NaN,1.380000,0,0,b,6950
1,2001,209,171,1,0,1.0,1.222222,0,1,a,21071
2,2001,115,91,1,1,2.0,1.263736,0,2,a,11591
3,2001,187,141,1,1,3.0,1.326241,0,3,b,18841
4,2001,154,105,1,0,4.0,1.466667,0,4,c,15505


## Creare una categoria con apply

La soluzione che abbiamo visto in precedenza per creare la categoria `tipo` ha due difetti: è fragile (perchè il valore soglia *1.3* compare in due istruzioni diverse, quindi è più probabile scrivere un valore sbagliato) e richiede un'istruzione distinta per ogni valore possibile di `tipo`.

Quindi prima scriviamo una funzione che riceve in input un numero e calcola il valore della categoria.

In [45]:
def categoria(n):
    soglie = [('a', 1.3), ('b', 1.4), ('c', None)]
    for (categoria, soglia) in soglie:
        if soglia is None or n < soglia:
            return categoria

Poi è possibile *applicare* la funzione a `rapporto`

In [46]:
incidenti['tipo'] = incidenti['rapporto'].apply(categoria)
incidenti.head(5)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo,pericolo
0,2001,69,50,1,1,NaN,1.380000,0,0,b,6950
1,2001,209,171,1,0,1.0,1.222222,0,1,a,21071
2,2001,115,91,1,1,2.0,1.263736,0,2,a,11591
3,2001,187,141,1,1,3.0,1.326241,0,3,b,18841
4,2001,154,105,1,0,4.0,1.466667,0,4,c,15505


## Differenze

Nelle slide precedenti abbiamo visto due diverse tipologie di utilizzo di apply:

1.  `incidenti.apply(calcola_p, axis = 1)`
2.  `incidenti['rapporto'].apply(categoria)`

Nel caso 1 diventa essenziale l'opzione `axis = 1` che indica a pandas che la funzione deve essere invocata per ogni riga. In questo caso la funzione `calcola_p` riceve un argomento che è la riga di un DataFrame. Tecnicamente la riga è un dizionario con chiavi corrispondenti alle colonne del DataFrame.

Nel caso 2 invece la `apply` viene chiamata su una Serie. In questo caso la funzione `categoria` riceve un singolo valore in input e la parte `axis = 1` non è necessaria.

Un terzo caso prevede invece che la funzione riceva un intero DataFrame

## Calcolo percentuale

Adesso vogliamo aggiungere una colonna che contiene la percentuale del numero di incidenti in un certo mese rispetto a tutti quelli dell'anno nella zona di riferimento. Questo calcolo richiede più passi.

Tramite un raggruppamento, possiamo calcolare il numero totale di incidenti per ogni anno e zona.

In [47]:
incidenti.groupby(['Anno', 'Zona_int']).sum()['Incidenti']

Anno  Zona_int
2001  0            605
      1           2207
      2           1171
      3           2176
      4           1782
      5           1693
      6           1409
      7           1828
      8           2200
      9           2387
2002  0            685
      1           2177
      2           1272
      3           2185
      4           1713
      5           1609
      6           1475
      7           1774
      8           2147
      9           2330
2003  0            671
      1           2056
      2           1160
      3           2061
      4           1623
      5           1628
      6           1371
      7           1868
      8           2034
      9           2247
                  ... 
2014  0            673
      1           1039
      2            567
      3           1008
      4            833
      5            839
      6            778
      7            944
      8           1090
      9           1069
2015  0            938
      1            

## Tabella pivot

Una tabella pivot viene ottenuta a partire da un DataFrame per riassumere i dati di quest'ultimo.
In particolare, in una tabella pivot sono presenti tre elementi:
*  un insieme di variabili per guidano il raggruppamento di righe
*  una variabile (o un insieme di variabili) che etichetta nuove colonne
*  una funzione per aggregare i valori.

Ad esempio, andiamo a costruire una tabella pivot che riassume per ogni *anno* il numero totale (quindi la *somma) di incidenti (feriti, ecc.), ed in cui si ha una colonna per ogni *zona*.

In [48]:
pd.pivot_table(incidenti, index = 'Anno', columns = ['Zona_int'], values = 'Incidenti', aggfunc = sum)

Zona_int,0,1,2,3,4,5,6,7,8,9
Anno,,,,,,,,,,
2001,605,2207,1171,2176,1782,1693,1409,1828,2200,2387
2002,685,2177,1272,2185,1713,1609,1475,1774,2147,2330
2003,671,2056,1160,2061,1623,1628,1371,1868,2034,2247
2004,682,1958,1101,2155,1579,1623,1287,1650,2051,2230
2005,716,1867,1023,1880,1479,1512,1232,1558,1898,2016
2006,675,1809,1011,1821,1527,1397,1204,1496,1996,2016
2007,574,1723,974,1735,1383,1432,1302,1495,1802,1956
2008,498,1502,916,1715,1322,1309,1132,1425,1747,1763
2009,494,1552,844,1507,1266,1232,1120,1308,1560,1647


## Tabella pivot con medie

Talvolta la funzione per aggregare i valori non è la somma, ma la media. In questo caso si può omettere l'opzione `aggfunc).

In [49]:
pd.pivot_table(incidenti, index = 'Anno', columns = ['Zona_int'], values = 'Incidenti')

Zona_int,0,1,2,3,4,5,6,7,8,9
Anno,,,,,,,,,,
2001,50.416667,183.916667,97.583333,181.333333,148.500000,141.083333,117.416667,152.333333,183.333333,198.916667
2002,57.083333,181.416667,106.000000,182.083333,142.750000,134.083333,122.916667,147.833333,178.916667,194.166667
2003,55.916667,171.333333,96.666667,171.750000,135.250000,135.666667,114.250000,155.666667,169.500000,187.250000
2004,56.833333,163.166667,91.750000,179.583333,131.583333,135.250000,107.250000,137.500000,170.916667,185.833333
2005,59.666667,155.583333,85.250000,156.666667,123.250000,126.000000,102.666667,129.833333,158.166667,168.000000
2006,56.250000,150.750000,84.250000,151.750000,127.250000,116.416667,100.333333,124.666667,166.333333,168.000000
2007,47.833333,143.583333,81.166667,144.583333,115.250000,119.333333,108.500000,124.583333,150.166667,163.000000
2008,41.500000,125.166667,76.333333,142.916667,110.166667,109.083333,94.333333,118.750000,145.583333,146.916667
2009,41.166667,129.333333,70.333333,125.583333,105.500000,102.666667,93.333333,109.000000,130.000000,137.250000


I dati presenti in una tabella pivot possono essere calcolati con una `groupby`, ma la struttura del DataFrame risultante è diversa. Ogni volta bisogna capire se preferiamo la forma ottenuta con la `groupby` (poche colonne, tante righe) o la tabella pivot (meno righe, più colonne).

Di solito la tabella pivot è più semplice da leggere per una persona, ma più complicata da rielaborare.

## Elementi estremi

Di particolare interesse è capire chi ha realizzato un valore estremo (per valore estremo si intende un minimo o un massimo). Ad esempio vogliamo sapere in quale anno si sono verificati più incidenti.

Il primo passo è calcolare il numero di incidenti per ogni anno.

In [50]:
incidenti_anno = incidenti.groupby('Anno').sum()[['Incidenti', 'Feriti', 'Morti']]
incidenti_anno.head(3)

,Incidenti,Feriti,Morti
Anno,,,
2001,17458,23998,86
2002,17367,23843,76
2003,16719,22422,72


Adesso possiamo calcolare il valore massimo della colonna `Incidenti` e selezionare le righe che hanno tale valore.

In [51]:
massimo = incidenti_anno['Incidenti'].max()
massimo

17458

In [52]:
incidenti_anno[incidenti_anno['Incidenti'] == massimo]

,Incidenti,Feriti,Morti
Anno,,,
2001,17458,23998,86


## Elementi estremi 2

Un'alternativa più rapida è fornita dai metodi `idxmax` e `idxmin` che restituiscono il valore dell'*indice* che realizza il valore estremo. Questo valore può essere utilizzato per estrarre la riga che ci interessa.

In [53]:
incidenti_anno['Incidenti'].idxmax()

2001

In [54]:
incidenti_anno.loc[incidenti_anno['Incidenti'].idxmax()]

Incidenti    17458
Feriti       23998
Morti           86
Name: 2001, dtype: int64

**Confronto**

*  La soluzione con `idxmax` è più veloce da scrivere e permette di individuare facilmente l'anno che ci interessa. Ma bisogna ricordare che `idxmax` permette di calcolare un solo valore (in questo caso, un solo anno).
*  La soluzione con la selezione è più complicata. Inoltre per calcolare l'anno che ci interessa, dovremmo fare una nuova estrazione. Il vantaggio è che è corretta anche nel caso in cui due anni diversi realizzano entrambi il massimo valore.

## Idxmax e raggruppamenti

Talvolta ci interessa capire chi ha realizzato un massimo all'interno di ogni gruppo. In altre parole, vogliamo isolare non una sola riga, ma una riga per ogni gruppo.

Ad esempio, per ogni anno vogliamo calcolare in quale mese si sono verificati più incidenti. Il primo passo è costruire un DataFrame che dica per ogni coppia (`Anno`, `Mese`) quanti incidenti si sono verificati.

In [55]:
incidenti_mese = incidenti.groupby(['Anno', 'Mese'], as_index = False).sum()[['Anno', 'Mese', 'Incidenti']]
incidenti_mese

,Anno,Mese,Incidenti
0,2001,1,1233
1,2001,2,1188
2,2001,3,1419
3,2001,4,1369
4,2001,5,1697
5,2001,6,1842
6,2001,7,1657
7,2001,8,848
8,2001,9,1491
9,2001,10,1675


Applicando `idxmax` otteniamo il singolo mese in cui si sono verificati più incidenti, ma noi vogliamo un mese per ogni anno (quindi un mese per il 2001, un mese per il 2002, ecc).

In [56]:
incidenti_mese.loc[incidenti_mese['Incidenti'].idxmax()]

Anno         2001
Mese            6
Incidenti    1842
Name: 5, dtype: int64

Quindi sappiamo che il picco del numero di incidenti si è verificato nel mese di Giugno del 2001. Ma ciò non è quello che desideriamo.

## Idxmax e raggruppamento 2

Raggruppare per anno le righe di `incidenti_mese` è il primo passo: bisogna applicare la funzione `idxmax`.

In [57]:
incidenti_mese.groupby('Anno')['Incidenti'].idxmax()

Anno
2001      5
2002     21
2003     28
2004     45
2005     52
2006     69
2007     76
2008     93
2009    105
2010    112
2011    130
2012    141
2013    153
2014    160
2015    177
2016    189
Name: Incidenti, dtype: int64

Siccome `idxmax` restituisce i valori dell'indice corrispondenti alle righe che realizzano un massimo, bisogna passare per una `loc` per ottenere i mesi corrispondenti.

In [58]:
incidenti_mese.loc[incidenti_mese.groupby('Anno')['Incidenti'].idxmax(), ['Anno', 'Mese']]

,Anno,Mese
5,2001,6
21,2002,10
28,2003,5
45,2004,10
52,2005,5
69,2006,10
76,2007,5
93,2008,10
105,2009,10
112,2010,5
